# Sentinel-1 two-date change detection with log-ratio

This notebook:

1. Uses the Copernicus **STAC API** to find the nearest Sentinel-1 GRD acquisitions
   to two target dates over a point of interest.
2. Downloads the corresponding **VV (or VH)** images via the **openEO** backend,
   clipped to a small bounding box around that point.
3. Computes a **log-ratio change image**:

   \[
   \log_{10}\left(\frac{\text{newer image}}{\text{older image}}\right)
   \]

4. Shows:
   - The **older** and **newer** images in Matplotlib (quick sanity check).
   - The **log-ratio** as a greyscale overlay on an interactive Folium map.

You can tweak:
- Location,
- Bounding box size,
- Dates,
- Polarisation (VV/VH),
- Output folder.


In [ ]:
# Imports (run once per environment)

# If you need to install packages, uncomment as needed:
# !pip install openeo rasterio folium pillow requests

import openeo
from pathlib import Path

import numpy as np
import rasterio

import matplotlib.pyplot as plt
import folium
from PIL import Image

import requests
from datetime import datetime, timedelta


## 1. Experiment parameters

Edit the values in this cell to change:

- `user_latitude`, `user_longitude`: centre of your area of interest.
- `buffer_degrees`: half-width of the bounding box (in degrees).
- `older_date`, `newer_date`: target dates for change detection.
- `polarisation`: `"VV"` or `"VH"`.
- `output_folder`: where all GeoTIFFs/PNGs will be saved.

The “date finder” will locate the **nearest actual Sentinel-1 GRD acquisition**
to each target date within a configurable ±days window.


In [ ]:
# ==== Experiment parameters (edit these for future runs) ====

# Centre of area of interest (decimal degrees)
user_latitude  = 37.38       # <-- change
user_longitude = -5.99       # <-- change

# Half-size of the bounding box in degrees
buffer_degrees = 0.1         # <-- change

# Two target dates for change detection (ISO format)
older_date = "2025-11-01"    # <-- change
newer_date = "2025-11-25"    # <-- change

# Sentinel-1 collection and band (polarisation)
collection_id = "SENTINEL1_GRD"
polarisation  = "VV"         # or "VH"

# Where to save outputs
output_folder = Path(r"C:\GIS\Learning SAR\S1_downloads")
output_folder.mkdir(parents=True, exist_ok=True)

# File paths will be defined later, AFTER we know the actual acquisition dates
older_path    = None
newer_path    = None
logratio_path = None
logratio_png  = None



## 2. Helper functions

Here we define:

- `get_spatial_extent`: converts centre + `buffer_degrees` into a bounding box dict
  that openEO expects (in EPSG:4326 lat/lon).
- `find_nearest_s1_acquisition`: queries the Copernicus STAC API to find the nearest
  Sentinel-1 GRD acquisition to a target date over the AOI, filtered by:
  - instrument mode (IW),
  - polarisation (VV or VH).

This avoids the “empty tiles” problem when no acquisition exists on the *exact* date.


In [ ]:
STAC_SEARCH_URL = "https://stac.dataspace.copernicus.eu/v1/search"

def get_spatial_extent(lat: float, lon: float, buffer_deg: float) -> dict:
    """Return an openEO-style bbox dict from centre + buffer (in degrees)."""
    return {
        "west":  lon - buffer_deg,
        "south": lat - buffer_deg,
        "east":  lon + buffer_deg,
        "north": lat + buffer_deg,
    }

def find_nearest_s1_acquisition(
    target_date_str: str,
    lat: float,
    lon: float,
    days_window: int = 10,
    polarization: str = "VV",
    instrument_mode: str = "IW",
    relative_orbit: int | None = None,
    orbit_state: str | None = None,   # "ascending" / "descending"
    platform: str | None = None,      # "sentinel-1a" / "sentinel-1b"
) -> dict:
    """
    Find the nearest Sentinel-1 GRD acquisition to a target date over a point.

    Returns a dict with:
      - 'datetime' (ISO string with Z)
      - 'relative_orbit' (int)
      - 'orbit_state' ("ascending"/"descending")
      - 'platform' (e.g. "sentinel-1a")
    """
    target_dt = datetime.fromisoformat(target_date_str)  # naive

    start = (target_dt - timedelta(days=days_window)).strftime("%Y-%m-%dT00:00:00Z")
    end   = (target_dt + timedelta(days=days_window)).strftime("%Y-%m-%dT23:59:59Z")

    payload = {
        "collections": ["sentinel-1-grd"],
        "datetime": f"{start}/{end}",
        "limit": 100,
        "intersects": {
            "type": "Point",
            "coordinates": [lon, lat],
        },
    }

    r = requests.post(STAC_SEARCH_URL, json=payload)
    r.raise_for_status()
    data = r.json()
    items = data.get("features", [])

    if not items:
        raise ValueError(f"No Sentinel-1 GRD items found near {target_date_str} for this AOI.")

    best_item = None
    best_dt = None
    best_diff = None

    for item in items:
        props = item.get("properties", {})

        # Instrument mode
        if instrument_mode is not None:
            if props.get("sar:instrument_mode") != instrument_mode:
                continue

        # Polarisation
        pols = props.get("sar:polarizations") or []
        if polarization not in pols:
            continue

        # Relative orbit filter (if given)
        rel = props.get("sat:relative_orbit")
        if relative_orbit is not None and rel != relative_orbit:
            continue

        # Orbit state filter (if given)
        state = props.get("sat:orbit_state")
        if orbit_state is not None and state != orbit_state:
            continue

        # Platform filter (if given)
        plat = props.get("platform")
        if platform is not None and plat != platform:
            continue

        dt_str = props.get("datetime")
        if not dt_str:
            continue

        try:
            dt = datetime.fromisoformat(dt_str.replace("Z", ""))  # naive
        except Exception:
            continue

        diff = abs((dt - target_dt).total_seconds())
        if best_diff is None or diff < best_diff:
            best_diff = diff
            best_dt = dt
            best_item = item

    if best_item is None:
        raise ValueError(
            f"No S1 GRD items matching filters near {target_date_str} in ±{days_window} days."
        )

    props = best_item["properties"]
    out = {
        "datetime": best_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "relative_orbit": props.get("sat:relative_orbit"),
        "orbit_state": props.get("sat:orbit_state"),
        "platform": props.get("platform"),
    }

    print(
        f"Target {target_date_str} → {out['datetime']} | "
        f"rel_orbit={out['relative_orbit']} | "
        f"orbit_state={out['orbit_state']} | "
        f"platform={out['platform']}"
    )
    return out


## 3. Connect to openEO and build spatial/temporal extents

This step:

1. Connects to the Copernicus Data Space openEO backend.
2. Authenticates you using OIDC.
3. Builds the spatial extent from your centre + buffer.
4. Uses `find_nearest_s1_acquisition` to get:
   - `older_datetime`, `newer_datetime` (actual acquisition times).
5. Creates one-point temporal extents for each image.


In [ ]:
# Connect to Copernicus Data Space openEO backend
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

print("Connected and authenticated.")

# Spatial extent in EPSG:4326
spatial_extent = get_spatial_extent(user_latitude, user_longitude, buffer_degrees)
print("Spatial extent:", spatial_extent)

# 1) Find older acquisition with basic filters (mode + pol)
older_info = find_nearest_s1_acquisition(
    target_date_str=older_date,
    lat=user_latitude,
    lon=user_longitude,
    days_window=10,
    polarization=polarisation,
    instrument_mode="IW",
)

older_datetime    = older_info["datetime"]
older_rel_orbit   = older_info["relative_orbit"]
older_orbit_state = older_info["orbit_state"]
older_platform    = older_info["platform"]

# 2) Find newer acquisition constrained to SAME track, direction, and platform
newer_info = find_nearest_s1_acquisition(
    target_date_str=newer_date,
    lat=user_latitude,
    lon=user_longitude,
    days_window=10,
    polarization=polarisation,
    instrument_mode="IW",
    relative_orbit=older_rel_orbit,
    orbit_state=older_orbit_state,
    platform=older_platform,
)

newer_datetime = newer_info["datetime"]

temporal_extent_older = [older_datetime, older_datetime]
temporal_extent_newer = [newer_datetime, newer_datetime]

print("Older temporal extent:", temporal_extent_older)
print("Newer temporal extent:", temporal_extent_newer)

# 3) Use ACTUAL acquisition dates (YYYY-MM-DD) in filenames
older_acq_date = older_datetime.split("T")[0]
newer_acq_date = newer_datetime.split("T")[0]

older_path    = output_folder / f"S1_{polarisation}_{older_acq_date}.tif"
newer_path    = output_folder / f"S1_{polarisation}_{newer_acq_date}.tif"
logratio_path = output_folder / f"S1_{polarisation}_logratio_{older_acq_date}_vs_{newer_acq_date}.tif"
logratio_png  = output_folder / f"S1_{polarisation}_logratio_{older_acq_date}_vs_{newer_acq_date}.png"

print("Older image path:   ", older_path)
print("Newer image path:   ", newer_path)
print("Log-ratio GeoTIFF:  ", logratio_path)
print("Log-ratio PNG:      ", logratio_png)



## 4. Download the two Sentinel-1 images

Here we:

1. Load Sentinel-1 GRD via openEO for:
   - The bounding box (`spatial_extent`),
   - Each temporal extent (`temporal_extent_older`, `temporal_extent_newer`),
   - The chosen polarisation (`bands=[polarisation]`).
2. (Optionally) reduce over time with a median, in case the window hits >1 acquisition
   (here each extent is a single datetime, so median is effectively a no-op).
3. Download each clipped image as a GeoTIFF to `older_path` and `newer_path`.


In [ ]:
# Older image
s1_older_cube = connection.load_collection(
    collection_id,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent_older,
    bands=[polarisation],
)

# If your extent ever spans multiple acquisitions, median is a sane reducer:
s1_older_cube = s1_older_cube.reduce_dimension(dimension="t", reducer="median")

# Reproject older cube to EPSG:4326 on the backend
s1_older_cube = s1_older_cube.resample_spatial(
    resolution=0.0001,      # ~10–11 m at the equator; tweak if you like
    projection=4326,        # EPSG:4326 (lat/lon)
    method="near",
)

s1_older_cube.download(str(older_path), format="GTiff")
print(f"Downloaded older image to: {older_path}")

# Newer image
s1_newer_cube = connection.load_collection(
    collection_id,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent_newer,
    bands=[polarisation],
)

s1_newer_cube = s1_newer_cube.reduce_dimension(dimension="t", reducer="median")

# Reproject newer cube to EPSG:4326 on the backend
s1_newer_cube = s1_newer_cube.resample_spatial(
    resolution=0.0001,
    projection=4326,
    method="near",
)

s1_newer_cube.download(str(newer_path), format="GTiff")
print(f"Downloaded newer image to: {newer_path}")


## 5. Quick stats and visual check (Matplotlib)

Before doing any change detection, it’s useful to check that:

- The rasters are not all zero.
- The values look plausible.
- The images look like proper SAR backscatter when stretched.

This cell:

- Computes simple stats (min, max, 2nd/98th percentiles).
- Shows each image with a percentile stretch in Matplotlib.

If Matplotlib is broken in your env, you can comment out the plotting bits
and just keep the stats.


In [ ]:
def quick_stats(path, title):
    with rasterio.open(path) as src:
        arr = src.read(1)
        print(f"\n=== {title} ===")
        print("Shape:", arr.shape, "dtype:", arr.dtype)
        print("Nodata value:", src.nodata)
        print("CRS:", src.crs)
        print("Transform:", src.transform)

        valid = np.isfinite(arr)
        if not np.any(valid):
            print("No finite pixels.")
            return

        arr_valid = arr[valid]
        print("Min:", float(arr_valid.min()), "Max:", float(arr_valid.max()))
        print("2nd percentile:", float(np.percentile(arr_valid, 2)))
        print("98th percentile:", float(np.percentile(arr_valid, 98)))

        nonzero = np.count_nonzero(arr_valid)
        total   = arr.size
        print(
            f"Non-zero pixels: {nonzero} / {total} "
            f"({100 * nonzero / total:.4f} %)"
        )

def show_stretched(path, title):
    with rasterio.open(path) as src:
        arr = src.read(1)

    valid = np.isfinite(arr) & (arr != 0)
    if not np.any(valid):
        print(f"{title}: no valid pixels to display.")
        return

    p2, p98 = np.percentile(arr[valid], [2, 98])

    plt.figure(figsize=(6, 6))
    plt.title(title)
    plt.imshow(arr, vmin=p2, vmax=p98, cmap="gray")
    plt.colorbar(label="Backscatter (stretched)")
    plt.axis("off")
    plt.show()

quick_stats(older_path, f"Older S1 {polarisation} (target {older_date})")
quick_stats(newer_path, f"Newer S1 {polarisation} (target {newer_date})")

show_stretched(older_path, f"Older S1 {polarisation}")
show_stretched(newer_path, f"Newer S1 {polarisation}")


## 6. Compute and save the log-ratio change image

We now compute:

\[
\text{logratio} = \log_{10}\left(\frac{\text{newer}}{\text{older}}\right)
\]

Implementation:

- Convert arrays to `float32`.
- Use `np.errstate` to suppress divide/invalid warnings.
- Where the older image is zero or non-finite, set the result to `NaN`.
- Save a georeferenced GeoTIFF (`logratio_path`) with `nodata = NaN`.

This is the raster you can also load in QGIS if you want.


In [ ]:
# Read older and newer arrays
with rasterio.open(older_path) as src_old:
    older_arr = src_old.read(1).astype("float32")
    profile = src_old.profile

with rasterio.open(newer_path) as src_new:
    newer_arr = src_new.read(1).astype("float32")

# Compute log10(newer/older)
with np.errstate(divide="ignore", invalid="ignore"):
    ratio = newer_arr / older_arr
    logratio = np.log10(ratio)

invalid_mask = (~np.isfinite(logratio)) | (older_arr == 0)
logratio[invalid_mask] = np.nan

# Save log-ratio GeoTIFF
log_profile = profile.copy()
log_profile.update(dtype="float32", count=1, nodata=np.nan)

with rasterio.open(logratio_path, "w", **log_profile) as dst:
    dst.write(logratio.astype("float32"), 1)

print(f"Log-ratio GeoTIFF saved to: {logratio_path}")


## 7. Show the log-ratio on a Folium map

To get an interactive map view:

1. Stretch the log-ratio values to 0–255 (2–98% percentile) and save as greyscale PNG.
2. Use the original `spatial_extent` (lat/lon) as the bounds for the overlay.
3. Display it on top of a Folium base map.

Note: we **do not** read bounds from the raster CRS (UTM 30N), because Folium expects
lat/lon (EPSG:4326). We simply reuse the AOI we originally requested.


In [ ]:
from datetime import datetime

# === 1) Acquisition + AOI + log-ratio stats ===

# Acquisition timing and orbit info
dt_old = datetime.fromisoformat(older_datetime.replace("Z", ""))
dt_new = datetime.fromisoformat(newer_datetime.replace("Z", ""))
delta_days = (dt_new - dt_old).total_seconds() / 86400.0

print("=== Sentinel-1 acquisition info ===")
print(f"Older acquisition : {older_datetime}")
print(f"Newer acquisition : {newer_datetime}")
print(f"Platform          : {older_platform}")
print(f"Relative orbit    : {older_rel_orbit}")
print(f"Orbit direction   : {older_orbit_state}")
print(f"Polarisation      : {polarisation}")
print(f"Time separation   : {delta_days:.2f} days")

# AOI / bbox info (requested extent)
west  = spatial_extent["west"]
south = spatial_extent["south"]
east  = spatial_extent["east"]
north = spatial_extent["north"]

centre_lat = (south + north) / 2
centre_lon = (west + east) / 2

print("\n=== Requested AOI (EPSG:4326) ===")
print(f"Centre lat, lon    : ({centre_lat:.6f}, {centre_lon:.6f})")
print(f"West / East (lon)  : {west:.6f}  /  {east:.6f}")
print(f"South / North (lat): {south:.6f} /  {north:.6f}")

# Log-ratio stats
valid = np.isfinite(logratio)
if np.any(valid):
    vals = logratio[valid]
    p2, p98 = np.nanpercentile(vals, [2, 98])
    print("\n=== Log10(newer/older) stats (valid pixels) ===")
    print(f"Min     : {float(np.nanmin(vals)):.4f}")
    print(f"Max     : {float(np.nanmax(vals)):.4f}")
    print(f"2nd pct : {float(p2):.4f}")
    print(f"98th pct: {float(p98):.4f}")
    print(f"Valid pixels: {vals.size} (out of {logratio.size})")
else:
    print("\n=== Log10(newer/older) stats ===")
    print("No valid (finite) log-ratio pixels.")
    p2, p98 = 0.0, 1.0  # fallback for stretch below

# === 2) Stretch logratio to 0–255 and save PNG ===
if np.any(valid):
    stretched = (logratio - p2) / (p98 - p2)
    stretched = np.clip(stretched, 0, 1)
    stretched[~valid] = 0
else:
    stretched = np.zeros_like(logratio, dtype="float32")

stretched_uint8 = (stretched * 255).astype("uint8")
img = Image.fromarray(stretched_uint8, mode="L")
img.save(logratio_png)
print(f"\nLog-ratio PNG saved to: {logratio_png}")

# === 3) Build Folium map using AOI bounds (lat/lon) ===
# We cannot transform UTM->WGS84 in this env, so we map the PNG to the requested AOI box.

m = folium.Map(location=[centre_lat, centre_lon], zoom_start=11, tiles=None)

# Base layer 1: OpenStreetMap
folium.TileLayer(
    tiles="OpenStreetMap",
    name="OpenStreetMap",
    control=True
).add_to(m)

# Base layer 2: Esri World Imagery (true-colour satellite)
folium.TileLayer(
    tiles="https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri World Imagery",
    name="Esri Satellite",
    overlay=False,
    control=True
).add_to(m)

# Log-ratio overlay (mapped onto the AOI box)
folium.raster_layers.ImageOverlay(
    name="Log10(newer/older)",
    image=str(logratio_png),
    bounds=[[south, west], [north, east]],  # AOI in lat/lon
    opacity=0.7,
    interactive=True,
    cross_origin=False,
).add_to(m)

folium.LayerControl().add_to(m)

m


## How to interpret the log-ratio change map

This map shows **log10(newer / older)** Sentinel-1 backscatter for the AOI. The overlay is a greyscale image:

### Mid-grey pixels (~0 after stretching)

- Little or no change in backscatter between the two dates.  
- Surfaces are scattering the radar in a similar way at both times.

### Bright pixels (positive log-ratio: newer > older)

Backscatter is **stronger in the newer image**.

Possible causes (context dependent):

- New built-up areas / infrastructure (more double-bounce).
- Vegetation removal / soil exposure (rougher surface).
- Drying of previously wet areas (less absorption, more reflection).
- Strong responses on steep slopes or tall structures where terrain effects are not fully corrected (we are using GRD, not full RTC).

### Dark pixels (negative log-ratio: newer < older)

Backscatter is **weaker in the newer image**.

Possible causes:

- Vegetation growth / canopy closure (more volume scattering, less surface return).
- Flooding or wetter surfaces (more absorption, smoother specular surfaces).
- Harvested crops / ploughed fields changing structure.
- Smoothing of rough surfaces (e.g. construction cleared, water level changes).

---

## Things to keep in mind

- This is **geometry-matched change**, not a land-cover classification:
  - A bright or dark patch is “change in scattering behaviour”, not automatically “urban expansion” or “deforestation”.
  - Interpretation always needs **context** (basemap, land cover, local knowledge).

- Very small, isolated bright/dark speckles can be **residual speckle noise** rather than real change.

- Because Sentinel-1 is **side-looking SAR** and the true-colour basemap is **near-nadir optical**, features will not line up perfectly in shape:
  - Buildings, tall objects and steep slopes can appear displaced or distorted in SAR due to layover and shadow, even when the geolocation is correct.
  - We are using GRD products (not full terrain-flattened RTC), so terrain-related distortions are still present.

- In this notebook specifically, the SAR change layer is mapped onto the **requested AOI box in lat/lon**, while the processing is done in UTM on a discrete grid. That snapping to the grid can introduce a small apparent shift or stretch relative to the optical basemap, even though the AOI is the same.

Use the Folium map interactively with the basemap (e.g. satellite or hybrid) to link bright/dark patches to real-world features and decide which changes are meaningful for your application.

